# ETL Pipeline of Youtube Video Transcripts

In [1]:
import requests   # Allows us to make API calls
import json   # Allows us to work w/ text in a JSON format
import pandas as pd
from datetime import datetime

# Import youtube API & my personal youtube API key
from my_sk import yt_api_key
from youtube_transcript_api import YouTubeTranscriptApi

# Import useful functions for ETL process
import utils

/Users/axel/Desktop/machine_learning/ShawTalebi_FSDS/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Extract Process : Extract Transcript Data of Shaw Talebi's YT Videos

In [2]:
# Define Channel ID of Shaw Talebi's YouTube channe;
channel_id = 'UCa9gErQ9AE5jT2DZLjXBIdA'

# Define the url for the API to use when you make a request
url = 'https://www.googleapis.com/youtube/v3/search'

# Initialize page token
page_token = None

# Initialize list to store video data
video_record_list = []

In [3]:
# Extract video data across multiple search result pages
while page_token != 0:
    # define parameters for API call
    params = {'key' : yt_api_key, 'channelId' : channel_id,
              'part' : ["snippet", "id"], 'order' : "date",
              'maxResults' : 50, 'pageToken' : page_token}
    
    # Make get request
    response = requests.get(url, params=params)
    video_record_list += utils.getVideoRecords(response)

    try:
        # Get next page token
        page_token = json.loads(response.text)["nextPageToken"]
    except:
        # If no next page token, kill while loop
        page_token = 0

In [4]:
# Store data in a Pandas DataFrame
df = pd.DataFrame(video_record_list)
df.head()

,video_id,datetime,title
0,bAe4qwQGxlI,2024-10-25T13:18:08Z,I Built an AI App in 4 days... here&#39;s how.
1,4QHg8Ix8WWQ,2024-10-17T12:50:12Z,Fine-Tuning BERT for Text Classification (Pyth...
2,tMiQIxSX64c,2024-10-10T13:50:57Z,5 AI Projects You Can Build This Weekend (with...
3,7Oy2NmPwJXo,2024-09-26T23:24:35Z,I Quit My Job… Here’s How Much I Made 1 Year L...
4,ZVVkdXHqEuM,2024-09-23T15:45:12Z,Knowledge Distillation Explained in 60 Seconds...


In [5]:
# Initialize a list to store video captions
transcript_text_list = []

# Loop through each row of videos dataframe
for i in range(len(df)):
    # Try to extract captions
    try:
        # get transcript
        transcript = YouTubeTranscriptApi.get_transcript(df['video_id'][i])
        transcript_text = utils.extract_text(transcript)
    except:
        # If no captions available set transcript text to "n/a"
        transcript_text = "n/a"
    # Append transcript text to list
    transcript_text_list.append(transcript_text)

In [6]:
df['transcript'] = pd.Series(transcript_text_list)
df.head()

,video_id,datetime,title,transcript
0,bAe4qwQGxlI,2024-10-25T13:18:08Z,I Built an AI App in 4 days... here&#39;s how.,I built a web app to translate YouTube videos ...
1,4QHg8Ix8WWQ,2024-10-17T12:50:12Z,Fine-Tuning BERT for Text Classification (Pyth...,massive Transformer models like GPT 40 llama a...
2,tMiQIxSX64c,2024-10-10T13:50:57Z,5 AI Projects You Can Build This Weekend (with...,the best way to develop your AI skills is by b...
3,7Oy2NmPwJXo,2024-09-26T23:24:35Z,I Quit My Job… Here’s How Much I Made 1 Year L...,14 months ago I made a big life change and lef...
4,ZVVkdXHqEuM,2024-09-23T15:45:12Z,Knowledge Distillation Explained in 60 Seconds...,knowledge distillation explained in 60 seconds...


## Transform Process : Clean & Transform Data into a Useable form for the ML Model

In [7]:
# Check for duplicates
print("shape : ", df.shape)
print("num unqiue rows : ", df.shape[0] - sum(df.duplicated()))
for column in df.nunique().index:
    print(f"num unique entries in {column} column : ", df.nunique()[column])

shape :  (109, 4)
num unqiue rows :  109
num unique entries in video_id column :  109
num unique entries in datetime column :  109
num unique entries in title column :  109
num unique entries in transcript column :  107


In [8]:
# Check Datatypes
print(f"Datatypes before : \n{df.dtypes}")

# Convert to correct datatype
#df['datetime'] = df.datetime.apply(lambda x : datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))

# Check Datatypes
print("Datatypes after : \n", df.dtypes)

Datatypes before : 
video_id      object
datetime      object
title         object
transcript    object
dtype: object
Datatypes after : 
 video_id      object
datetime      object
title         object
transcript    object
dtype: object


In [9]:
# List special characters and their replacements
special_strings = ['&#39;', '&amp;', 'sha ']
special_string_replacements = ["'", '&', 'Shaw ']

# Replace each special string that appears in title and transcript columns
for i in range(len(special_strings)):
    df['title'] = df.title.apply(lambda x : x.replace(special_strings[i], special_string_replacements[i]))
    df['transcript'] = df.transcript.apply(lambda x : x.replace(special_strings[i], special_string_replacements[i]))

In [10]:
df.head()

,video_id,datetime,title,transcript
0,bAe4qwQGxlI,2024-10-25T13:18:08Z,I Built an AI App in 4 days... here's how.,I built a web app to translate YouTube videos ...
1,4QHg8Ix8WWQ,2024-10-17T12:50:12Z,Fine-Tuning BERT for Text Classification (Pyth...,massive Transformer models like GPT 40 llama a...
2,tMiQIxSX64c,2024-10-10T13:50:57Z,5 AI Projects You Can Build This Weekend (with...,the best way to develop your AI skills is by b...
3,7Oy2NmPwJXo,2024-09-26T23:24:35Z,I Quit My Job… Here’s How Much I Made 1 Year L...,14 months ago I made a big life change and lef...
4,ZVVkdXHqEuM,2024-09-23T15:45:12Z,Knowledge Distillation Explained in 60 Seconds...,knowledge distillation explained in 60 seconds...


## Load Process : Load the clean & transformed data into the project's 'data' directory


In [18]:
# Save data as parquet file
df.to_parquet('data/video-transcripts.parquet')